In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from sklearn.model_selection import train_test_split
import joblib
import tensorflow as tf
import keras_tuner
import json

In [2]:
df=pd.read_csv('D:\\DL\\ann_model_regression_energy_effecency\\data\\processed\\energy_efficiency.csv')
df.head()

,Relative_Compactness,Surface_Area,Wall_Area,Roof_Area,Overall_Height,Glazing_Area,Heating_Load
0,0.98,514.5,294.0,110.25,7.0,0.0,15.55
1,0.98,514.5,294.0,110.25,7.0,0.0,15.55
2,0.98,514.5,294.0,110.25,7.0,0.0,15.55
3,0.98,514.5,294.0,110.25,7.0,0.0,15.55
4,0.90,563.5,318.5,122.50,7.0,0.0,20.84


In [4]:
x=df.drop(columns=["Heating_Load"])
y=df["Heating_Load"]

In [15]:
x.shape,y.shape

((768, 6), (768,))

In [5]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [6]:
scaller=StandardScaler()
x_train=scaller.fit_transform(x_train)
x_test=scaller.transform(x_test)
x_train,x_test

(array([[ 0.55367107, -0.6962224 , -0.00737214, -0.67904812,  1.01642146,
         -1.01029954],
        [-1.15511771,  1.25066366,  0.55843925,  0.95706259, -0.98384385,
          1.22779036],
        [ 0.93340191, -0.97434898, -0.57318352, -0.67904812,  1.01642146,
          0.10874541],
        ...,
        [-0.49058874,  0.41628392, -1.13899491,  0.95706259, -0.98384385,
         -1.01029954],
        [ 2.07259443, -1.80872872, -0.57318352, -1.49710347,  1.01642146,
          0.10874541],
        [ 1.31313275, -1.25247556, -0.00737214, -1.22441835,  1.01642146,
         -1.01029954]], shape=(614, 6)),
 array([[-1.34498313,  1.52879024,  1.12425064,  0.95706259, -0.98384385,
          1.22779036],
        [-0.96525229,  0.97253708, -0.00737214,  0.95706259, -0.98384385,
          0.10874541],
        [ 2.07259443, -1.80872872, -0.57318352, -1.49710347,  1.01642146,
          1.22779036],
        [ 0.26887294, -0.41809582,  0.55843925, -0.67904812,  1.01642146,
          1.22779036],

In [16]:
def build_model(hp):
    model= tf.keras.Sequential()
    counter=0
    for i in range(hp.Int('num_layers',1,5)):
        if counter==0:
            model.add(tf.keras.layers.Input(shape=(6, )))
            model.add(tf.keras.layers.Dense(hp.Int('units_'+str(i), min_value=32, max_value=128, step=16), activation=hp.Choice('activation_'+str(i), values=['relu', 'tanh'])))
            model.add(tf.keras.layers.Dropout(hp.Float('dropout_'+str(i), min_value=0.0, max_value=0.5, step=0.1)))
        else:
            model.add(tf.keras.layers.Dense(hp.Int('units_'+str(i), min_value=32, max_value=128, step=16), activation=hp.Choice('activation_'+str(i), values=['relu', 'tanh'])))
            model.add(tf.keras.layers.Dropout(hp.Float('dropout_'+str(i), min_value=0.0, max_value=0.5, step=0.1)))
        counter+=1
    model.add(tf.keras.layers.Dense(1, activation='linear')) 
    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop']), loss='mean_squared_error', metrics=['mae'])
    return model
       

In [17]:
tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5)

In [18]:
tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test),verbose=1)

Trial 5 Complete [00h 00m 03s]
val_loss: 55.144012451171875

Best val_loss So Far: 9.37106990814209
Total elapsed time: 00h 00m 17s


In [19]:
tuner.results_summary()

Results summary
Results in .\untitled_project
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 1 summary
Hyperparameters:
num_layers: 4
units_0: 128
activation_0: tanh
dropout_0: 0.1
optimizer: adam
units_1: 80
activation_1: tanh
dropout_1: 0.1
units_2: 32
activation_2: relu
dropout_2: 0.0
units_3: 32
activation_3: relu
dropout_3: 0.0
Score: 9.37106990814209

Trial 0 summary
Hyperparameters:
num_layers: 2
units_0: 32
activation_0: relu
dropout_0: 0.0
optimizer: adam
units_1: 32
activation_1: relu
dropout_1: 0.0
Score: 37.49559783935547

Trial 4 summary
Hyperparameters:
num_layers: 2
units_0: 128
activation_0: tanh
dropout_0: 0.0
optimizer: adam
units_1: 64
activation_1: tanh
dropout_1: 0.1
units_2: 112
activation_2: relu
dropout_2: 0.0
units_3: 96
activation_3: tanh
dropout_3: 0.0
Score: 55.144012451171875

Trial 3 summary
Hyperparameters:
num_layers: 3
units_0: 112
activation_0: tanh
dropout_0: 0.1
optimizer: rmsprop
units_1: 32
activation_1: tanh
dropout_1: 0

In [21]:
tuning_params=tuner.get_best_hyperparameters()[0].values

In [24]:
with open("D:\\DL\\ann_model_regression_energy_effecency\\tuning_params\\best_hyperparameters.json", "w") as f:
    json.dump(tuning_params, f, indent=False)